#### Extend data
无论zvt外的数据是什么格式，我们都使用zvt简单统一的契约来定义它们。

In [ ]:
from sqlalchemy import Column, String
from sqlalchemy.orm import declarative_base
from zvt.contract import Mixin
from zvt.contract.register import register_schema
NewsBase = declarative_base()
class StockNews(NewsBase, Mixin):
    __tablename__ = "stock_news"
    #: news title
    news_title = Column(String)

register_schema(providers=["em"], db_name="stock_news", schema_base=NewsBase, entity_type="stock")

In [ ]:
StockNews.get_storages()

In [ ]:
# 如果你有其他提供商，比如新浪，也注册一下
register_schema(providers=["em", "sina"], db_name="stock_news", schema_base=NewsBase, entity_type="stock")
# 你可以找到另一个db文件
StockNews.get_storages()

#### Implement recorder
实现新的record, StockNews

In [ ]:
import pandas as pd
from zvt.contract.api import df_to_db
from zvt.contract.recorder import FixedCycleDataRecorder
from zvt.domain import Stock
from zvt.domain.misc.stock_news import StockNews
from zvt.recorders.em import em_api


class EMStockNewsRecorder(FixedCycleDataRecorder):
    original_page_url = "https://wap.eastmoney.com/quote/stock/0.002572.html"
    url = "https://np-listapi.eastmoney.com/comm/wap/getListInfo?cb=callback&client=wap&type=1&mTypeAndCode=0.002572&pageSize=200&pageIndex={}&callback=jQuery1830017478247906740352_1644568731256&_=1644568879493"

    # the entity class you record for
    entity_schema = Stock
    # connect schema with recorder
    data_schema = StockNews
    # entity data provider
    entity_provider = "em"
    # data schema provider
    provider = "em"

    def record(self, entity, start, end, size, timestamps):
        news = em_api.get_news(entity_id=entity.id)
        df = pd.DataFrame.from_records(news)
        self.logger.info(df)
        df_to_db(df=df, data_schema=self.data_schema, provider=self.provider, force_update=self.force_update)

In [ ]:
StockNews.record_data(provider="em", entity_id="stock_sz_002572")
df = StockNews.query_data(entity_id="stock_sz_002572")
print(df.head(10))

In [ ]:
#### 添加新的TradableEntity

In [ ]:
##### Define entity Schema

In [ ]:
# -*- coding: utf-8 -*-

from sqlalchemy import Column, String, Float
from sqlalchemy.orm import declarative_base

from zvt.contract.schema import TradableEntity
from zvt.contract.register import register_schema, register_entity

CountryMetaBase = declarative_base()


@register_entity(entity_type="country")
class Country(CountryMetaBase, TradableEntity):
    __tablename__ = "country"

    #: 区域
    #: region
    region = Column(String(length=128))
    #: 首都
    #: capital city
    capital_city = Column(String(length=128))
    #: 收入水平
    #: income level
    income_level = Column(String(length=64))
    #: 贷款类型
    #: lending type
    lending_type = Column(String(length=64))
    #: 经度
    #: longitude
    longitude = Column(Float)
    #: 纬度
    #: latitude
    latitude = Column(Float)


register_schema(providers=["wb"], db_name="country_meta", schema_base=CountryMetaBase)

In [ ]:
entity_type = "country"
exchange = "galaxy"
code = "iso code"

In [ ]:
from zvt.contract.api import df_to_db
from zvt.contract.recorder import Recorder
from zvt.domain.meta.country_meta import Country
from zvt.recorders.wb import wb_api


class WBCountryRecorder(Recorder):
    provider = "wb"
    data_schema = Country

    def run(self):
        df = wb_api.get_countries()
        df_to_db(df=df, data_schema=self.data_schema, provider=self.provider, force_update=self.force_update)

In [ ]:
# -*- coding: utf-8 -*-
from sqlalchemy import Column, String, Float
from sqlalchemy.orm import declarative_base

from zvt.contract import Mixin
from zvt.contract.register import register_schema

CurrencyBase = declarative_base()


class TreasuryYield(CurrencyBase, Mixin):
    __tablename__ = "treasury_yield"

    code = Column(String(length=32))

    # 2年期
    yield_2 = Column(Float)
    # 5年期
    yield_5 = Column(Float)
    # 10年期
    yield_10 = Column(Float)
    # 30年期
    yield_30 = Column(Float)


register_schema(providers=["em"], db_name="currency", schema_base=CurrencyBase)

In [ ]:
from zvt.domain import Country
Country.record_data()
df = Country.query_data()
df[df['income_level']=='High income']

In [ ]:
from zvt.domain import TreasuryYield
from zvt.api.intent import compare
TreasuryYield.record_data()
compare(codes=["US"], schema=TreasuryYield, columns=["yield_2", "yield_10", "yield_30"])